## Imports 

In [2]:
import numpy as np
import pandas as pd

## Constants 

In [3]:
TRAIN_TEXT_PATH = './SpamData/02_Training/train-data.txt'
TEST_TEXT_PATH = './SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB = './SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB = './SpamData/03_Testing/prob-ham.txt'
TOKEN_ALL_PROB = './SpamData/03_Testing/prob-all.txt'

TEST_FEATURE_MATRIX = './SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = './SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

## Load features from .txt files to numpy array 

In [4]:
sparse_train = np.loadtxt(TRAIN_TEXT_PATH, delimiter=' ', dtype=int)
sparse_test = np.loadtxt(TEST_TEXT_PATH, delimiter=' ', dtype=int)

In [5]:
sparse_test[:5]

array([[ 8,  7,  1,  1],
       [ 8,  8,  1,  1],
       [ 8, 12,  1,  1],
       [ 8, 19,  1,  2],
       [ 8, 26,  1,  1]])

In [6]:
sparse_test[-5:]

array([[5793,  833,    0,    1],
       [5793, 1090,    0,    1],
       [5793, 1125,    0,    1],
       [5793, 1271,    0,    1],
       [5793, 2003,    0,    1]])

In [7]:
print('Nr of unique doc ids in train data ', np.unique(sparse_train[:, 0]).size)
print('Nr of unique doc ids in test data ', np.unique(sparse_test[:, 0]).size)

Nr of unique doc ids in train data  4015
Nr of unique doc ids in test data  1724


In [8]:
print('Nr of rows in train data ', sparse_train.shape[0])
print('Nr of rows in test data ', sparse_test.shape[0])

Nr of rows in train data  265475
Nr of rows in test data  110523


## Create an empty dataframe 

In [9]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))

In [10]:
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
index_names = np.unique(sparse_train[:, 0])

In [12]:
full_matrix = pd.DataFrame(index=index_names, columns=column_names)
full_matrix.fillna(value=0, inplace=True)

In [13]:
full_matrix.tail()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def create_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Returns a pandas dataframe.
    Key arguments:
    
    sparse_matrix -- numpy array 
    nr_words -- size of the vocabulary 
    doc_id -- position of doc_id in sparse matrix. Default 1st position
    word id -- position of word id in sparse matrix. Default 2nd position 
    category -- position of category in sparse matrix (Spam 1 , non-spam 0). Default 3rd position 
    frequency -- position of occurence of word in sparse matrix. Default 4th position
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, nr_words))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        
        doc_id = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        frequency = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_id, 'DOC_ID'] = doc_id
        full_matrix.at[doc_id, 'CATEGORY'] = label
        full_matrix.at[doc_id, word_id] = frequency
    
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix 

In [15]:
%%time 

full_train_data = create_full_matrix(sparse_train, VOCAB_SIZE)

CPU times: user 11 s, sys: 207 ms, total: 11.2 s
Wall time: 12 s


In [16]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,0,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,2,2,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5791,0,2,0,1,1,0,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,2,2,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5795,0,1,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
full_train_data.CATEGORY.sum()

1250

## Training Naive Bayes Classifier 

In [19]:
# prob_spam = full_train_data[full_train_data.CATEGORY == 1].shape[0] / full_train_data.shape[0]
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam ', prob_spam)

Probability of spam  0.31133250311332505


## Total number of Words / Tokens

In [20]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']

In [21]:
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
email_lengths = full_train_features.sum(axis=1)

In [23]:
total_words = email_lengths.sum()

In [24]:
total_words

445898

## Number of tokens in Spam and Ham 

In [25]:
spam_features = full_train_data.loc[full_train_data.CATEGORY == 1, full_train_data.columns != 'CATEGORY']

In [26]:
spam_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [27]:
spam_lengths = spam_features.sum(axis=1)

In [28]:
total_words_spam = spam_lengths.sum()

In [29]:
ham_features = full_train_data.loc[full_train_data.CATEGORY == 0, full_train_data.columns != 'CATEGORY']

In [30]:
ham_lengths = ham_features.sum(axis=1)

In [31]:
total_words_ham = ham_lengths.sum()

In [32]:
print('Total words in Spam emails ', total_words_spam)
print('Total words in normal emails ', total_words_ham)

Total words in Spam emails  195641
Total words in normal emails  250257


In [33]:
print('Avg number of words in spam emails {:.0f}'.format(total_words_spam / spam_lengths.shape[0]))
print('Avg number of words in normal emails {:.3f}'.format(total_words_ham / ham_lengths.shape[0]))

Avg number of words in spam emails 157
Avg number of words in normal emails 90.509


## Summed token dataframe 

In [34]:
spam_token_sum = spam_features.sum(axis=0) + 1

In [35]:
spam_token_sum

0       1793
1        968
2       1355
3       2099
4       1348
        ... 
2495       7
2496       1
2497       9
2498       5
2499       2
Length: 2500, dtype: int64

In [36]:
ham_token_sum = ham_features.sum(axis=0) + 1

In [37]:
ham_token_sum

0       5273
1       2499
2       2038
3        944
4       1595
        ... 
2495      32
2496      22
2497      24
2498      29
2499      30
Length: 2500, dtype: int64

## P(Token | Spam) -- Probability that a token occurs given the email is Spam 

In [38]:
prob_tokens_spam = spam_token_sum / (total_words_spam + VOCAB_SIZE)

In [39]:
prob_tokens_spam

0       0.009049
1       0.004885
2       0.006839
3       0.010593
4       0.006803
          ...   
2495    0.000035
2496    0.000005
2497    0.000045
2498    0.000025
2499    0.000010
Length: 2500, dtype: float64

In [40]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) -- Probability that a token occurs given that email is Ham

In [41]:
prob_tokens_ham = ham_token_sum / (total_words_ham + VOCAB_SIZE)

In [42]:
prob_tokens_ham

0       0.020862
1       0.009887
2       0.008063
3       0.003735
4       0.006310
          ...   
2495    0.000127
2496    0.000087
2497    0.000095
2498    0.000115
2499    0.000119
Length: 2500, dtype: float64

## P(Token)  -- Probability that token occurs

In [43]:
prob_tokens_all = full_train_features.sum(axis=0) / total_words

In [45]:
prob_tokens_all.min()

4.485330725861071e-06

## Saving the trained model 

In [46]:
np.savetxt(TOKEN_SPAM_PROB, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB, prob_tokens_all)

## Prepare test data

In [47]:
sparse_test.shape

(110523, 4)

In [48]:
%%time 

full_test_data = create_full_matrix(sparse_test, nr_words=VOCAB_SIZE)

CPU times: user 4.3 s, sys: 57.2 ms, total: 4.36 s
Wall time: 4.36 s


In [49]:
full_test_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
12,1,0,2,1,1,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
15,1,3,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
19,1,1,2,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,0,2,2,2,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5786,0,2,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5788,0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [50]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY 

In [51]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)

In [53]:
prob_tokens_all.min()

4.485330725861071e-06